In [24]:
#####################################################################################################
#####################################################################################################

##  ATTENTION

~~we trained a bert model on the SMILES stuctures and their approval lables. here is the result of the model~~

~~Add the performance of the embedding model to the prompt~~

~~array([0.93596226]) handle this similarity score. remove the array()~~

~~similarity not simmilarity~~

~~add names , the approval status(Groups) of the drug in the table.~~

do the inference embeding for all moleculs and add the results to the table. 

Sample 10 or 20 small moleculs that are approved/non approved for prompt tuning randomly. 

In [23]:
#####################################################################################################
#####################################################################################################

In [3]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw, AllChem, Lipinski, Crippen, rdMolDescriptors
from rdkit.Chem import rdDepictor
import pandas as pd

def get_molecule_properties(smiles):
    """
    Compute molecular properties and analyses for a given SMILES string.
    
    Args:
        smiles (str): SMILES string of the molecule.
    
    Returns:
        dict: A dictionary containing all computed properties and analyses.
    """
    # Initialize the molecule
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        raise ValueError("Invalid SMILES string provided.")
    
    # Dictionary to store all properties
    properties = {}
    

    properties["Molecular_Weight"] = Descriptors.MolWt(molecule)
    
    # 2. Molecular Descriptors
    properties["LogP"] = Crippen.MolLogP(molecule)
    properties["Molecular_Refractivity"] = Crippen.MolMR(molecule)
    properties["TPSA"] = Descriptors.TPSA(molecule)
    properties["Hydrogen_Bond_Donors"] = Lipinski.NumHDonors(molecule)
    properties["Hydrogen_Bond_Acceptors"] = Lipinski.NumHAcceptors(molecule)
    properties["Rotatable_Bonds"] = Lipinski.NumRotatableBonds(molecule)
    
    
    # 4. Stereochemistry
    properties["Chiral_Centers"] = Chem.FindMolChiralCenters(molecule)
    
    
    # # 6. Substructure Searching
    # substructure = Chem.MolFromSmarts("CO")  # Search for C-O bond
    # properties["Substructure_Matches"] = molecule.GetSubstructMatches(substructure)
    
    return properties

In [1]:
import pandas as pd 
final_df = pd.read_csv("/home/mamreza/Desktop/LLM-DrugApprovalPredictor/samples/train_reason_sample.csv")
final_df['status'] = final_df['labels'].map({1: 'Approved', 0 :'Not Approved'})

In [67]:
df = pd.read_csv("/home/mamreza/Desktop/LLM-DrugApprovalPredictor/dataframes/final_df_to_reason.csv")


In [4]:
final_df["rdkit_info"] = final_df["SMILES"].apply(lambda x: get_molecule_properties(x))

In [6]:
sys_prompt = "you are an expert chemist "

sys_prompt2 = """You are an expert computational chemist specializing in drug discovery and molecular modeling. You have extensive knowledge of medicinal chemistry, pharmacokinetics, molecular docking, QSAR modeling, and structure-activity relationships (SAR). Your task is to analyze chemical compounds and predict their likelihood of being viable drug candidates based on their molecular structure, physicochemical properties, and bioavailability. Consider key factors such as Lipinski’s Rule of Five, ADMET (absorption, distribution, metabolism, excretion, toxicity) properties, target binding affinity, and known pharmacological data when making your assessments. Provide scientifically sound explanations for your predictions."""

In [92]:
sys_prompt3 = """You are an expert  chemist specializing in drug discovery and molecular modeling. You have extensive knowledge of medicinal chemistry, pharmacokinetics, molecular docking, QSAR modeling, and structure-activity relationships (SAR). Your task is to analyze chemical compounds and predict their likelihood of being viable drug candidates based on their molecular structure, physicochemical properties, and bioavailability. Consider key factors such as :
1. **Lipinski’s Rule of Five**: Assess whether the compound adheres to the rule (molecular weight ≤ 500, LogP ≤ 5, hydrogen bond donors ≤ 5, hydrogen bond acceptors ≤ 10).
2. **ADMET Properties**: Evaluate absorption, distribution, metabolism, excretion, and toxicity profiles, including:
   - **Absorption**: Solubility, permeability (e.g., Caco-2, PAMPA assays).
   - **Distribution**: Volume of distribution, plasma protein binding.
   - **Metabolism**: Cytochrome P450 enzyme interactions, metabolic stability.
   - **Excretion**: Renal clearance, biliary excretion.
   - **Toxicity**: Hepatotoxicity, cardiotoxicity, genotoxicity, and Ames test predictions.
4. **Veber’s Rule**: Assess drug-likeness based on polar surface area (PSA ≤ 140 Å²) and number of rotatable bonds (≤ 10).
5. **Ghose Filter**: Evaluate drug-likeness using criteria such as LogP (-0.4 to 5.6), molecular weight (160–480 Da), molar refractivity (40–130), and total atoms (20–70).
6. **Egan’s Rule**: Predict oral bioavailability using LogP (≤ 5.88) and topological polar surface area (TPSA ≤ 131.6 Å²).
7. **PAINS (Pan-Assay Interference Compounds)**: Identify and flag compounds with substructures known to cause assay interference or nonspecific binding.
8. **Brenk Filters**: Identify problematic functional groups that may lead to toxicity or poor pharmacokinetics.
9. **Rule of Three (for lead-like compounds)**: Assess compounds for lead optimization (molecular weight ≤ 300, LogP ≤ 3, hydrogen bond donors ≤ 3, hydrogen bond acceptors ≤ 3, rotatable bonds ≤ 3).
10. **Pharmacophore Modeling**: Evaluate whether the compound fits the essential features of a pharmacophore model for the target.
11. **QSAR Models**: Use quantitative structure-activity relationship models to predict biological activity and physicochemical properties.
12. **Bioavailability and Bioactivity**: Predict oral bioavailability and bioactivity using machine learning models and experimental data.
13. **Structural Alerts**: Identify structural motifs associated with toxicity or metabolic instability.
14. **Solubility and Permeability**: Use predictive models like the Biopharmaceutics Classification System (BCS) to assess solubility and permeability.
15. **DrugBank and ChEMBL Data**: Cross-reference known pharmacological data and target interactions from databases like DrugBank and ChEMBL."""

In [93]:
model_performance = """
{'F1 score': 0.80,
 'Accuracy': 0.69,
 'Precision': 0.68,
 'Recall': 0.975,
 'Specificity': 0.20}
 """
#the model's performance is {model_performance} 'AUC': 0.5892699115044248,


In [153]:
# n = 3 intersting 
n = 2

## Prompts

In [154]:
prompt4 = f"""
{sys_prompt3}

I have developed a model that can predict the drug likelibility of compound X.
This model outputs two lists:  
1. The five most similar approved small molecules (with their properties and similarity scores).  
2. The five most similar non-approved small molecules (with their properties and similarity scores).  

Your task is to analyze the likelihood of compound X receiving regulatory approval based on its similarity to known molecules.  

- RDKit Analysis of Compound X:
{final_df["rdkit_info"][n]}

Using the provided data:  
- Approved molecules (with similarity scores): {final_df["list_of_most_approved_info_for_n_simmilar"][n]}  
- Non-approved molecules (with similarity scores): {final_df["list_of_most_nonapproved_info_for_n_simmilar"][n]}  
"""
###############################

In [155]:
prompt4

'\nYou are an expert  chemist specializing in drug discovery and molecular modeling. You have extensive knowledge of medicinal chemistry, pharmacokinetics, molecular docking, QSAR modeling, and structure-activity relationships (SAR). Your task is to analyze chemical compounds and predict their likelihood of being viable drug candidates based on their molecular structure, physicochemical properties, and bioavailability. Consider key factors such as :\n1. **Lipinski’s Rule of Five**: Assess whether the compound adheres to the rule (molecular weight ≤ 500, LogP ≤ 5, hydrogen bond donors ≤ 5, hydrogen bond acceptors ≤ 10).\n2. **ADMET Properties**: Evaluate absorption, distribution, metabolism, excretion, and toxicity profiles, including:\n   - **Absorption**: Solubility, permeability (e.g., Caco-2, PAMPA assays).\n   - **Distribution**: Volume of distribution, plasma protein binding.\n   - **Metabolism**: Cytochrome P450 enzyme interactions, metabolic stability.\n   - **Excretion**: Renal

In [156]:
second_prompt = f"""based on the previous reasoning Think step by step and explain why it is {final_df["status"][n]}"""

second_prompt2 = f"""But wait the true answer is {final_df["status"][n]}. think step by step and tell why"""
## DANGEROUS:  YOU NEED TO DELETE THE FIRST THINKING PARAGRAPH OF THIS 

In [157]:
second_prompt2

'But wait the true answer is Approved. think step by step and tell why'

In [158]:
third_prompt = f"""
write your thinking in the following format 
{{"status(Approved/NonApproved)": {final_df["status"][n]}, 
"reason":   }}
"""

In [152]:
third_prompt

'\nwrite your thinking in the following format \n{"status(Approved/NonApproved)": Approved, \n"reason":   }\n'

In [118]:
prompt4

'\nYou are an expert  chemist specializing in drug discovery and molecular modeling. You have extensive knowledge of medicinal chemistry, pharmacokinetics, molecular docking, QSAR modeling, and structure-activity relationships (SAR). Your task is to analyze chemical compounds and predict their likelihood of being viable drug candidates based on their molecular structure, physicochemical properties, and bioavailability. Consider key factors such as :\n1. **Lipinski’s Rule of Five**: Assess whether the compound adheres to the rule (molecular weight ≤ 500, LogP ≤ 5, hydrogen bond donors ≤ 5, hydrogen bond acceptors ≤ 10).\n2. **ADMET Properties**: Evaluate absorption, distribution, metabolism, excretion, and toxicity profiles, including:\n   - **Absorption**: Solubility, permeability (e.g., Caco-2, PAMPA assays).\n   - **Distribution**: Volume of distribution, plasma protein binding.\n   - **Metabolism**: Cytochrome P450 enzyme interactions, metabolic stability.\n   - **Excretion**: Renal

In [15]:
prompt5 = f"""
{sys_prompt2}

I have developed a model that can predict the drug likelibility of compound X.
This model outputs two lists:  
1. The five most similar approved small molecules (with their properties and similarity scores).  
2. The five most similar non-approved small molecules (with their properties and similarity scores).  

Your task is to analyze the likelihood of compound X receiving regulatory approval based on its similarity to known molecules.  

Using the provided data:  
- Approved molecules (with similarity scores): {final_df["list_of_most_approved_info_for_n_simmilar"][n]}  
- Non-approved molecules (with similarity scores): {final_df["list_of_most_nonapproved_info_for_n_simmilar"][n]}  
please predict that will the compound X approved? 

and fill the following dictionary
{{"status": }}

answer = {final_df["status"][n]}
"""

In [16]:
prompt5

'\nYou are an expert computational chemist specializing in drug discovery and molecular modeling. You have extensive knowledge of medicinal chemistry, pharmacokinetics, molecular docking, QSAR modeling, and structure-activity relationships (SAR). Your task is to analyze chemical compounds and predict their likelihood of being viable drug candidates based on their molecular structure, physicochemical properties, and bioavailability. Consider key factors such as Lipinski’s Rule of Five, ADMET (absorption, distribution, metabolism, excretion, toxicity) properties, target binding affinity, and known pharmacological data when making your assessments. Provide scientifically sound explanations for your predictions.\n\nI have developed a model that can predict the drug likelibility of compound X.\nThis model outputs two lists:  \n1. The five most similar approved small molecules (with their properties and similarity scores).  \n2. The five most similar non-approved small molecules (with their 

In [ ]:
########### ADD THIS TO THE END OF PROMPT   

"""
and fill the following dictionary
{"status": }
\n\n"

answer = non approved
"""

In [11]:
prompt4

'\nYou are an expert computational chemist specializing in drug discovery and molecular modeling. You have extensive knowledge of medicinal chemistry, pharmacokinetics, molecular docking, QSAR modeling, and structure-activity relationships (SAR). Your task is to analyze chemical compounds and predict their likelihood of being viable drug candidates based on their molecular structure, physicochemical properties, and bioavailability. Consider key factors such as Lipinski’s Rule of Five, ADMET (absorption, distribution, metabolism, excretion, toxicity) properties, target binding affinity, and known pharmacological data when making your assessments. Provide scientifically sound explanations for your predictions.\n\nI have developed a model that can predict the drug likelibility of compound X.\nThis model outputs two lists:  \n1. The five most similar approved small molecules (with their properties and similarity scores).  \n2. The five most similar non-approved small molecules (with their 

## Prompt arxives

In [17]:


input_prompt_3 = f"""
Your task is to analyze the likelihood of compound X receiving regulatory approval based on its similarity to known molecules.  

I have developed a model that identifies molecules most similar to compound X. This model outputs two lists:  
1. The five most similar approved small molecules (with their properties and similarity scores).  
2. The five most similar non-approved small molecules (with their properties and similarity scores).  

Using the provided data:  
- Approved molecules (with similarity scores): {final_df["list_of_most_approved_info_for_n_simmilar"][n]}  
- Non-approved molecules (with similarity scores): {final_df["list_of_most_nonapproved_info_for_n_simmilar"][n]}  

Please analyze the molecular properties, trends, and distinguishing features in both lists. Take the similarity scores into account when evaluating the degree of resemblance between compound X and the approved/non-approved molecules.  

Based on these comparisons, determine whether compound X is more likely to be approved or not. Justify your reasoning by highlighting key characteristics that align with approved or non-approved compounds, and discuss how the similarity scores influence your prediction.  

"""


In [18]:
input_prompt_3

'\nYour task is to analyze the likelihood of compound X receiving regulatory approval based on its similarity to known molecules.  \n\nI have developed a model that identifies molecules most similar to compound X. This model outputs two lists:  \n1. The five most similar approved small molecules (with their properties and similarity scores).  \n2. The five most similar non-approved small molecules (with their properties and similarity scores).  \n\nUsing the provided data:  \n- Approved molecules (with similarity scores): [{\'name\': None, \'simmilarity score\': array([0.99735814]), \'approval status\': None, \'average mass\': None, \'toxicity\': None, \'descriptions\': None}, {\'name\': \'Budesonide\', \'simmilarity score\': array([0.99513477]), \'approval status\': \'approved\', \'average mass\': 430.5339, \'toxicity\': \'Acute overdose of corticosteroids is rare, however prolonged high dosing of corticosteroids can lead to hypercorticism and adrenal axis suppression.[L10601,L10604,L

In [ ]:
input_prompt_2 = f"""{sys_prompt}

Your task is to provide a detailed reasoning on why compound X has an approval status of {final_df["status"][n]}.

I have developed a model that identifies molecules most similar to compound X. This model outputs two lists:
1. The five most similar approved small molecules (with their properties).
2. The five most similar non-approved small molecules (with their properties).

Using the provided data:
- Approved molecules: {final_df["list_of_most_approved_info_for_n_simmilar"][n]}
- Non-approved molecules: {final_df["list_of_most_nonapproved_info_for_n_simmilar"][n]}
"""

In [ ]:
prompt = f"""
Text: {text}
Label: {label}
Generate a short reasoning for why the label is appropriate.
Reasoning: 
"""

In [ ]:
input_prompt_2

In [ ]:
input_prompt_3

In [ ]:
prompt7 = f"""
{sys_prompt3}

**Drug Approval Prediction Task for Compound X**

**Model Context:**
- Trained RoBERTa classifier for SMILES molecules (Approved/Non-approved)
- Similarity search performed using final layer embeddings
- Data sources: DrugBank (properties/status), RDKit (compound analysis)

**Provided Data for Compound X:**
1. RDKit Analysis:
{final_df["rdkit_info"][n]}

2. Most Similar Approved Compounds (Top 5):
{final_df["list_of_most_approved_info_for_n_simmilar"][n]}

3. Most Similar Non-Approved Compounds (Top 5):
{final_df["list_of_most_nonapproved_info_for_n_simmilar"][n]}

**Prediction Task:**
Analyze compound X's approval potential through systematic evaluation:

1. **Similarity Analysis:**
   - Compare similarity scores between approved/non-approved groups
   - Identify patterns in structural/chemical similarities
   - Note any outlier similarities

2. **Property Comparison:**
   - Cross-reference RDKit properties with both groups
   - Highlight matching properties with approved drugs
   - Flag properties aligning with non-approved compounds

3. **Model Confidence:**
   - Consider model performance metrics in evaluation
   - Account for potential false positives/negatives
   - Evaluate confidence level based on similarity distributions

**Required Output Format:**
"**Approval Prediction:** [Approved/Non-Approved] (Confidence: High/Medium/Low)

**Rationale:**
1. Key similarity factors: [List 2-3 main structural/property similarities]
2. Differentiating features: [List 1-2 crucial differences]
3. Model considerations: [Note any accuracy/precision/recal limitations]
4. Supporting evidence: [Specific examples from similar compounds]

**Recommended Next Steps:**
[Suggest 2-3 further analyses or verifications]"
"""

In [ ]:
prompt6 = f"""


{sys_prompt2}


I have trained a roberta model for classification of molecules in SMILES format to approved and non-approved. The prformace of the model is: 
{model_performance}
I used the last layer of the model to embedd SMILES and the did similarity search for each smiles. For each SMILES I returned the 5 most similaars and 5 most nonsimilars with their properties and similarity scores and  their properties. 
I used Drugbank to get all properties. 
I also used RDKit to get information for my molecul. 
your task is to predict the drug approval status of compund X. 

Using the provided data:  
- information from RDKit that I got: {final_df["rdkit_info"][0]}
- Approved molecules (with similarity scores): {final_df["list_of_most_approved_info_for_n_simmilar"][n]}  
- Non-approved molecules (with similarity scores): {final_df["list_of_most_nonapproved_info_for_n_simmilar"][n]}  
Think step by step and predict that will the compound X approved? 
"""

## API building dataset

In [ ]:
#########  https://api-docs.deepseek.com/guides/reasoning_model
from openai import OpenAI
client = OpenAI(api_key="<DeepSeek API Key>", base_url="https://api.deepseek.com")

# Round 1
messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

# Round 2
messages.append({'role': 'assistant', 'content': content})
messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)
# ...